In [1]:
exec(open('init_notebook.py').read())

current working dirD:\ML\nlp\nlp-practice


In [2]:
import wget
import pandas as pd
import numpy as np
import numpy.typing as npt
import nltk
import seaborn as sns
import tqdm
import os
from nltk.corpus import stopwords
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.metrics import RocCurveDisplay, roc_curve
from sklearn.metrics import accuracy_score, balanced_accuracy_score

from nltk import word_tokenize
%load_ext autoreload
%autoreload 2

In [4]:
dataDir = os.path.join(projectFolder, "data")
dataDir

'D:/ML/nlp/nlp-practice\\data'

In [5]:
# url1 = "https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt"
# url2 = "https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt"
# wget.download(url1, out=dataDir)
# wget.download(url2, out=dataDir)

# Task = Given a Poem, who is the poet?
## Model using a Bayes-Classifier.

p(x|author1), p(x|author2) can be calculated by creating two markov models, one for author1 and one for author2. So, we can use unsupervised method for supervised task!

In [6]:
from library.common_preprocess import *
from library.selection import *
import math

In [7]:
with open(os.path.join(dataDir, "edgar_allan_poe.txt")) as f:
    poeSentences = f.readlines()
with open(os.path.join(dataDir, "robert_frost.txt")) as f:
    frostSentences = f.readlines()

## Step 1: Lemmatize

In [8]:
poeSentences = lemmatize(poeSentences)
frostSentences = lemmatize(frostSentences)

## Step 2: train-test split

In [9]:
poTrain, poTest = split(poeSentences, 0.7)
frostTrain, frostTest = split(frostSentences, 0.7)

## Step 3: Build vocabulary based on the train sets

In [10]:
trainSet = poTrain + frostTrain

In [11]:
wordToIndex = buildWordToIndex (
    word_tokenize, 
    trainSet, 
    maxSize = 5000,
    # stopWords=stopwords.words('english'),
    stopWords=None,
    lowercase=True,
    ignorePunkt=True
)
idxToWord = {v: k for k, v in wordToIndex.items()}

*** top 10 words out of 1958 words***
the 599
and 306
be 301
i 280
a 275
of 268
to 263
in 209
it 179
that 157


In [12]:
vocabularySize = len(wordToIndex)
vocabularySize

1959

## Step 4: tokenize 

In [13]:
poTrainTokens = [tokenizeDoc(word_tokenize, doc, wordToIndex) for doc in poTrain]
frostTrainTokens = [tokenizeDoc(word_tokenize, doc, wordToIndex) for doc in frostTrain]

In [14]:
poTrainTokens[0]

[7, 4, 233, 557, 15, 420]

In [15]:
poTrain[0]

'In a strange city , all alone ,'

## Step 5: Build Markov Models

In [16]:
TransitionMatrix = npt.NDArray
MarginalProbabilities = npt.NDArray
Probablity = float

# Markov Model for an author
def buildMarkovModel(sentences: List[List[Token]], vocabularySize: int) -> Tuple[TransitionMatrix, MarginalProbabilities]:
    # step 1: initialize A, and pi
    
    # Initialize with Add-One Smoothing or a very low value
    pi = np.ones(vocabularySize)
    A = np.ones((vocabularySize, vocabularySize))
    
    # print(vocabularySize)
    for aSenTokens in sentences:
        # print(aSenTokens)
        if len(aSenTokens) > 0:
            pi[aSenTokens[0]] += 1 # initial state
            for i in range(1, len(aSenTokens)): # i next, j is previous.
                iToken = aSenTokens[i]
                jToken = aSenTokens[i-1]
                A[iToken, jToken] += 1
                
    # now each column in A represents a previous word. So, we normalize the column to get probabilities P(i|j).
    
    pi /= np.sum(pi)
    A /= np.sum(A, axis=0)
    
    # followin is the validation check if each column adds up to 1
    # Asums = np.sum(A, axis=0)
    # ones = np.ones_like(Asums)
    # for col in range(len(Asums)):
    #     assert np.allclose(Asums, ones)
    
    return A, pi



In [17]:
poA, poPi = buildMarkovModel(poTrainTokens, vocabularySize)
frA, frPi = buildMarkovModel(frostTrainTokens, vocabularySize)

## Step 6: Bayes Model - Generative Text

In [18]:
# It does not work well as it just learns bi-grams.

class GenModel:
    def __init__(self, 
                 indexToWord: Dict[Token, WordRoot],  
                 A1: TransitionMatrix, 
                 pi1: MarginalProbabilities, 
                 A2: TransitionMatrix, 
                 pi2: MarginalProbabilities
        ):
        self.indexToWord = indexToWord
        
        self.mdps = {
            1: [A1, pi1],
            2: [A2, pi2]
        }
        
    
    def __call__(self, label: Label, n: int) -> Text:
        # 1. sample n tokens
        # 2. convert tokens to words
        
        A, pi = self.mdps[label]
        allTokens = list(range(len(poPi)))
        
        # 1. 
        txtBuffer = []
        firstToken = np.random.choice(allTokens, p=poPi)
        txtBuffer.append(self.indexToWord[firstToken]) # our first word root!
        
        prevToken = firstToken
        for i in range(1, n):
            nextTokenProbs = A[:, prevToken]
            nextToken = np.random.choice(allTokens, p=nextTokenProbs)
            txtBuffer.append(self.indexToWord[nextToken])
        return " ". join(txtBuffer)
            
        
            
        
    

In [19]:
model = GenModel(
        indexToWord=idxToWord,
        A1=poA,
        pi1=poPi,
        A2=frA,
        pi2=frPi
    )

In [20]:
poLabel = 1
frLabel = 2

In [29]:
for seed in range(3):
    np.random.seed(seed)
    sonnet = [model(poLabel, 10) for _ in range(14)]
    print("*******seed************************************", seed)
    print("\n".join(sonnet))

*******seed************************************ 0
his glory birth spirit spirit birth spirit undivided wandering spirit
is owe owe waste the the the waste waste waste
pauses in in in in in in in in in
in tear her some the good good good couple thine
that night hang the hang hang i be on my
where no israfel the the the deep the no the
i be journey i see know wake be roam be
blushes with with with with with with with with with
it may be duplicate grow not own be grow ash
it doth be duplicate not be grow be thrive be
thy memory holy peer memory presence presence presence memory peer
they follow trail be follow would plunge trail plunge trail
through the two an the an the the an the
their eternal bed hymn echoing merry echoing melancholy melancholy eternal
*******seed************************************ 1
but guide the they to a that where now light
not leave the linger the like that with the the
these rose cheek rose rose rose cheek cheek cheek rose
and now wisdom star palace not palace c

# Step 7: Evaluate

In [22]:
# Train

In [23]:
# build the confusion matrix
Y_pred = []
Y_test = []
for sen in poTest:
    pred = model(sen)
    if pred is None:
        pred = -1
    Y_pred.append(pred)
    Y_test.append(poLabel)
    
for sen in frostTest:
    pred = model(sen)
    if pred is None:
        pred = -1
    Y_pred.append(pred)
    Y_test.append(frLabel)
    
cm = confusion_matrix(Y_test, Y_pred)
disp = ConfusionMatrixDisplay(cm, display_labels=["None", "Poe", "Frost"])
disp.plot()
print(accuracy_score(Y_test, Y_pred), balanced_accuracy_score(Y_test, Y_pred))

TypeError: __call__() missing 1 required positional argument: 'n'

In [ ]:
# build the confusion matrix
Y_pred = []
Y_test = []
for sen in poTrain:
    pred = model(sen)
    if pred is None:
        pred = -1
    Y_pred.append(pred)
    Y_test.append(poLabel)
    
for sen in frostTrain:
    pred = model(sen)
    if pred is None:
        pred = -1
    Y_pred.append(pred)
    Y_test.append(frLabel)
    
cm = confusion_matrix(Y_test, Y_pred)
disp = ConfusionMatrixDisplay(cm, display_labels=["None", "Poe", "Frost"])
disp.plot()
print(accuracy_score(Y_test, Y_pred), balanced_accuracy_score(Y_test, Y_pred))
